In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [2]:
# cluster_accuracy = 65  # 0-100 (100 = very tight clusters, but higher percentage of no_cluster groups)
# min_cluster_size = 2  # set the minimum size of cluster groups. (Lower number = tighter groups)


# accuracy = [70, 80]
# size = [2, 8]
 
# results = []
# for a in accuracy:
#     for s in size:
#         results.append({'Accuracy' : a, 'Minimum cluster size': s, 'Number of orphans': None})


# # On veut sélectionner les paramètres qui vont maximiser le accuracy et minimiser le nombre d'orphans
# results = pd.DataFrame(results)
# results

In [3]:
file_path = '../06-clustering/candidate_terms.csv'
with open(file_path, encoding='utf-8') as f:
    df = pd.read_csv(f).drop(columns=["Unnamed: 0"]).sample(1000)
    df['Terme'] = df['Terme'].astype('str')
    df['TF + DF'] = df['TF'] + df['DF']

# store the data
cluster_name_list = []
corpus_sentences_list = []
df_all = []

corpus_set = set(df['Terme'])
corpus_set_all = corpus_set
cluster = True

df

,Corpora,Terme,Structure syntaxique,Forme lemmatisée,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm,Log Likelihood,TF,DF,TF*IDF,OKapiBM25,TF + DF
58,['sante_mtl'],santé montréal accès aux espaces,NOM ADJ NOM PRP:det NOM,santé montréal accès aux espace,False,NaN,NaN,NaN,False,-inf,3046,3046,0.447214,15.885776,6092
5937,"['chuqc', 'inesss']",lettre d'autorisation,NOM PRP NOM,lettre de autorisation,False,NaN,NaN,NaN,False,1368.005310,87,83,0.195134,20.264013,170
10392,['ciusss_cn'],thiamine thiosulfate de sodium,NOM NOM PRP NOM,thiamine thiosulfate de sodium,False,NaN,NaN,NaN,False,833.337158,48,48,0.081609,24.387984,96
5095,['inesss'],inspq visitez,NOM VER:pres,inspq visiter,False,NaN,NaN,NaN,False,-inf,107,107,0.059649,6.695343,214
4645,['cisss_cotenord'],population de la côte-nord,NOM PRP DET:ART NOM,population de le côte-nord,False,NaN,NaN,NaN,False,1332.166812,122,121,0.357195,11.782546,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,['msss'],ventilation établissements sociosanitaires,NOM NOM ADJ,ventilation établissement sociosanitaires,False,NaN,NaN,NaN,False,-inf,172,172,0.134854,16.601194,344
5027,['msss'],portrait cumulatif depuis le début de l'année,NOM ADJ PRP DET:ART NOM PRP DET:ART NOM,portrait cumulatif depuis le début de le année,False,NaN,NaN,NaN,False,1436.070003,108,108,0.100534,8.938169,216
237,"['cisss_ca', 'cisss_cotenord', 'cisss_gaspesie...",type familial,NOM ADJ,type familial,False,NaN,NaN,NaN,False,1217.898360,1228,605,1.000000,34.006775,1833
529,"['ciusss_cn', 'inspq', 'quebec_sante']",centre antipoison,NOM ADJ,centre antipoison,False,NaN,NaN,NaN,True,623.113280,675,296,0.893326,21.400376,971


In [4]:
from nltk.tokenize import RegexpTokenizer
regex = "[\w+-]+|\([\s+\w+\d+-]+\)|\w+|\w"
tokenizex = RegexpTokenizer(regex)

file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stop = [t.lower().strip('\n') for t in f.readlines()]

def to_tokens(kw, min_chars=2):
    tokens = tokenizex.tokenize(str(kw)) # split the string into a list of words
    tokens = [word for word in tokens if len(word) > min_chars] 
    tokens = [str(word) for word in tokens if word not in stop] 
    
    tokens = set(tokens) # to remove duplicates
    tokens = sorted(tokens) # converts our set back to a list and sorts words in alphabetical order
    return tokens

df["tokens"] = df["Terme"].apply(lambda x: to_tokens(
    x,
    min_chars=3,
))
df

,Corpora,Terme,Structure syntaxique,Forme lemmatisée,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm,Log Likelihood,TF,DF,TF*IDF,OKapiBM25,TF + DF,tokens
58,['sante_mtl'],santé montréal accès aux espaces,NOM ADJ NOM PRP:det NOM,santé montréal accès aux espace,False,NaN,NaN,NaN,False,-inf,3046,3046,0.447214,15.885776,6092,"[accès, espaces, santé]"
5937,"['chuqc', 'inesss']",lettre d'autorisation,NOM PRP NOM,lettre de autorisation,False,NaN,NaN,NaN,False,1368.005310,87,83,0.195134,20.264013,170,"[autorisation, lettre]"
10392,['ciusss_cn'],thiamine thiosulfate de sodium,NOM NOM PRP NOM,thiamine thiosulfate de sodium,False,NaN,NaN,NaN,False,833.337158,48,48,0.081609,24.387984,96,"[sodium, thiamine, thiosulfate]"
5095,['inesss'],inspq visitez,NOM VER:pres,inspq visiter,False,NaN,NaN,NaN,False,-inf,107,107,0.059649,6.695343,214,"[inspq, visitez]"
4645,['cisss_cotenord'],population de la côte-nord,NOM PRP DET:ART NOM,population de le côte-nord,False,NaN,NaN,NaN,False,1332.166812,122,121,0.357195,11.782546,243,"[côte-nord, population]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,['msss'],ventilation établissements sociosanitaires,NOM NOM ADJ,ventilation établissement sociosanitaires,False,NaN,NaN,NaN,False,-inf,172,172,0.134854,16.601194,344,"[sociosanitaires, ventilation, établissements]"
5027,['msss'],portrait cumulatif depuis le début de l'année,NOM ADJ PRP DET:ART NOM PRP DET:ART NOM,portrait cumulatif depuis le début de le année,False,NaN,NaN,NaN,False,1436.070003,108,108,0.100534,8.938169,216,"[année, cumulatif, portrait]"
237,"['cisss_ca', 'cisss_cotenord', 'cisss_gaspesie...",type familial,NOM ADJ,type familial,False,NaN,NaN,NaN,False,1217.898360,1228,605,1.000000,34.006775,1833,"[familial, type]"
529,"['ciusss_cn', 'inspq', 'quebec_sante']",centre antipoison,NOM ADJ,centre antipoison,False,NaN,NaN,NaN,True,623.113280,675,296,0.893326,21.400376,971,"[antipoison, centre]"


In [5]:
# # keep looping through until no more clusters are created
# def community_clusters(cluster_accuracy, min_cluster_size, corpus_set=corpus_set):
#     cluster_accuracy = cluster_accuracy / 100
#     cluster = True
#     while cluster:
#         corpus_sentences = list(corpus_set)
#         check_len = len(corpus_sentences)

#         corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
#         clusters = util.community_detection(corpus_embeddings, min_community_size= min_cluster_size, threshold=cluster_accuracy, init_max_size=len(corpus_embeddings))

#         for keyword, cluster in enumerate(clusters):
#             print("\nCluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

#             for sentence_id in cluster[0:]:
#                 print("\t", corpus_sentences[sentence_id])
#                 corpus_sentences_list.append(corpus_sentences[sentence_id])
#                 cluster_name_list.append("Cluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

#         df_new = pd.DataFrame(None)
#         df_new['Cluster Name'] = cluster_name_list
#         df_new['Cluster Name'].fillna("zzz_no_cluster")

#         df_new["Terme"] = corpus_sentences_list

#         df_all.append(df_new)
#         have = set(df_new["Terme"])

#         corpus_set = corpus_set_all - have
#         remaining = len(corpus_set)
#         print("Total Unclustered Keywords: ", remaining)
#         if check_len == remaining:
#             break
#         return remaining

In [6]:
# for a in accuracy:
#     for s in size:
#         score = community_clusters(a, s)
#         results.loc[((results['Accuracy'] == a) & (results['Minimum cluster size'] == s)), \
#         'Number of orphans'] = score

In [7]:
# results

# # Ensuite, on voudrait représenter sur un graphique le accuracy, la taille des clusters et le nombre d'orphelins pour prendre
# # la valeur combinaison de valeurs maximisant l'accuracy et minimisant le nombre de clusters orphelins

# file_path = '../06-Clustering/results_BERT_communityDetection_sample.csv'
# results.to_csv(file_path)

In [8]:
# ########################################
# # En fonction du graphique, on choisit nos paramètres
# ########################################
# import seaborn as sns
# import numpy as np
# results['Number of orphans'] = pd.to_numeric(results['Number of orphans'])
# results_pivot = results.pivot(index= 'Accuracy', columns='Minimum cluster size', values = 'Number of orphans')
# results_pivot

# fig = sns.heatmap(results_pivot, annot=True, fmt="d", cmap='binary').get_figure()
# file_path = '../06-Clustering/figure_BERT_communityDetection.tiff'
# fig.savefig(file_path) 

In [9]:
############################
# Choix des paramètres

cluster_accuracy = 50
min_cluster_size = 2

############################

cluster_accuracy = cluster_accuracy / 100
cluster = True
while cluster:
    corpus_sentences = list(corpus_set)
    check_len = len(corpus_sentences)

    corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
    clusters = util.community_detection(corpus_embeddings, min_community_size= min_cluster_size, threshold=cluster_accuracy, init_max_size=len(corpus_embeddings))

    for keyword, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

        for sentence_id in cluster[0:]:
            print("\t", corpus_sentences[sentence_id])
            corpus_sentences_list.append(corpus_sentences[sentence_id])
            cluster_name_list.append("Cluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

    df_new = pd.DataFrame(None)
    df_new['Cluster Name'] = cluster_name_list
    df_new["Terme"] = corpus_sentences_list

    df_all.append(df_new)
    have = set(df_new["Terme"])

    corpus_set = corpus_set_all - have
    remaining = len(corpus_set)
    print("Total Unclustered Keywords: ", remaining)
    if check_len == remaining:
        break

Batches: 100%|██████████| 4/4 [00:34<00:00,  8.62s/it]



Cluster 1, #84 Elements 
	 changement pertinent
	 actions concrètes
	 cliniques spécialisées
	 changements majeurs
	 consultations spécialisées
	 drogues pluralité
	 déficience auditive
	 dose antérieure
	 visiteur journalier
	 médecine légale
	 hôpital neurologique
	 utérus persistant
	 examen serviront
	 planification globale
	 membres supérieurs
	 impacts négatifs
	 sujets variés
	 cellules présentant
	 usagers actifs
	 médecin traitant
	 documentation références tuberculose
	 garde éducatifs
	 délais prévus
	 tests rapides
	 réussite éducative
	 adolescence aire
	 résultats sources
	 lait maternel
	 guichets spécifiques
	 temps supplémentaire
	 problèmes liés
	 dose supplémentaire
	 recours conformément
	 études supérieures
	 données de manière
	 ensure plus formule
	 garde éducatif
	 gram négatif
	 onglets représentent
	 services de type
	 veilleurs du programme
	 sanis inscription
	 enfant âgé
	 trouble lié
	 aide sociale
	 génétique moléculaire
	 québec version
	 mois archives


Batches: 100%|██████████| 2/2 [00:14<00:00,  7.29s/it]



Cluster 1, #7 Elements 
	 transport des usagers cliniques médicales prise
	 usagers cliniques médicales
	 présentation des manifestations cliniques
	 recrutement de médecins
	 douleur chronique pratiques cliniques
	 effectifs médicaux (pem)
	 services de type clinique

Cluster 2, #6 Elements 
	 prélèvements santé mentale
	 santé mentale et psychiatrie
	 soins de collaboration en santé mentale
	 soins infirmiers en santé mentale
	 institut national de psychiatrie
	 santé mentale et dépendance

Cluster 3, #5 Elements 
	 milieu scolaire spécialisé réadaptation
	 services spécialisés de réadaptation
	 centre de réadaptation pour les jeunes en difficulté
	 centre de réadaptation pour les jeunes
	 naissances urgence réadaptation photos

Cluster 4, #4 Elements 
	 gestion des risques
	 traitements personnes à risques
	 facteurs de risques associés aux lésions
	 réduction du risque de conduite

Cluster 5, #4 Elements 
	 pré-ac en attente d'évaluation dojolvi triheptanoïne
	 demande en attente 

Batches: 100%|██████████| 2/2 [00:13<00:00,  6.63s/it]

Total Unclustered Keywords:  350


In [10]:
# make a new dataframe from the list of dataframe and merge back into the orginal df
df_new = pd.concat(df_all)
df = df.merge(df_new.drop_duplicates('Terme'), how='left', on="Terme")

df['Cluster Name'] = df['Cluster Name'].fillna("zzz_no_cluster")

In [11]:
# Attribuer un label significatif aux clusters 
# Deux options :
#   1 - S'il y a un token en commun  entre les termes d'un même clusters, le clusters sera désigné par celui-ci
#   2 - Sinon, on prend le terme dont la somme TF + DF est la plus élevée 
# rename the clusters to the shortest keyword in the cluster

labels = set(df['Cluster Name'].tolist())
desired_labels = {x : None for x in labels} # (on initialise à None)
for label in labels:
    d = df[df['Cluster Name'] == label]['tokens'].tolist()
    new_label = list(set.intersection(*map(set,d)))
    try:
        desired_labels[label] = new_label[0]
    except:
            cluster = df[df["Cluster Name"] == label]
            max_freq = cluster['TF + DF'].max()
            new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values
            desired_labels[label] = new_label[0]

df['Cluster'] = df['Cluster Name'].map(desired_labels)

In [12]:
# move the cluster and keyword columns to the front
col = df.pop("Terme")
df.insert(0, col.name, col)

col = df.pop('Cluster')
df.insert(0, col.name, col)

df.sort_values(["Cluster", "Terme"], ascending=[True, True], inplace=True)

In [13]:
df

,Cluster,Terme,Corpora,Structure syntaxique,Forme lemmatisée,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm,Log Likelihood,TF,DF,TF*IDF,OKapiBM25,TF + DF,tokens,Cluster Name
973,accès,guichet d'accès di-tsa-dp,"['ciusss_centresud', 'ciusss_ouestmtl', 'sante...",NOM PRP NOM NOM,guichet de accès di-tsa-dp,False,NaN,NaN,NaN,False,1368.003664,118,57,0.442942,10.176053,175,"[accès, di-tsa-dp, guichet]","Cluster 59, #3 Elements"
181,accès,langue anglaise guichet d'accès bilingue,['ciusss_cn'],NOM ADJ NOM PRP NOM ADJ,langue anglais guichet de accès bilingue,False,NaN,NaN,NaN,False,-inf,139,132,0.167088,7.715851,271,"[accès, anglaise, bilingue, guichet, langue]","Cluster 59, #3 Elements"
211,accès,services pour tous accès,['santeestrie'],NOM PRP PRO:IND NOM,service pour tout accès,False,NaN,NaN,NaN,False,-inf,142,142,0.086460,3.887422,284,"[accès, services]","Cluster 59, #3 Elements"
907,acétylcystéine,acétylcystéine acétylcystéine sandoz,['inesss'],NOM NOM VER:subi,acétylcystéine acétylcystéine sandoz,False,NaN,NaN,NaN,False,661.237731,39,39,0.008728,16.715233,78,"[acétylcystéine, sandoz]","Cluster 80, #2 Elements"
139,acétylcystéine,solution d'acétylcystéine acétylcystéine,['inesss'],NOM PRP NOM NOM,solution de acétylcystéine acétylcystéine,False,NaN,NaN,NaN,False,855.821992,42,42,0.008700,17.098753,84,"[acétylcystéine, solution]","Cluster 80, #2 Elements"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,évaluation cablivi caplacizumab,évaluation cablivi caplacizumab,['inesss'],NOM ADJ NOM,évaluation cablivi caplacizumab,False,NaN,NaN,NaN,False,1468.190986,66,66,0.007977,16.567989,132,"[cablivi, caplacizumab, évaluation]","Cluster 26, #5 Elements"
904,évaluation du réseau,mesure conférences,['ciusss_cn'],NOM NOM,mesure conférence,False,NaN,NaN,NaN,False,1041.571671,94,94,0.243417,11.820003,188,"[conférences, mesure]","Cluster 40, #4 Elements"
266,évaluation du réseau,mesure conférences formation,['ciusss_cn'],NOM NOM NOM,mesure conférence formation,False,NaN,NaN,NaN,False,1137.889996,82,82,0.250510,18.824362,164,"[conférences, formation, mesure]","Cluster 40, #4 Elements"
194,évaluation du réseau,mesure conférences formation réseau,['ciusss_cn'],NOM NOM NOM NOM,mesure conférence formation réseau,False,NaN,NaN,NaN,False,1237.009574,82,82,0.250510,22.784374,164,"[conférences, formation, mesure, réseau]","Cluster 40, #4 Elements"


In [14]:
file_path = '../06-clustering/clusters_BERT_communityDetection_sample.csv'
df.drop(columns=['Cluster Name', 'tokens']).to_csv(file_path, index=False)

In [15]:
uncluster_percent = (remaining / len(df)) * 100
clustered_percent = 100 - uncluster_percent
print(clustered_percent,"% of rows clustered successfully!")

65.0 % of rows clustered successfully!


## **Visualisation**

In [16]:
# On va d'abord choisir les clusters qu'on veut visualiser (ajouter une colonne = Visualize Y/N)
clusters = pd.DataFrame(set(df['Cluster'].tolist()), columns=['Cluster'])

file_path = '../06-clustering/clusters_to_visualize.csv'
clusters.to_csv(file_path)

__________________________________________________________________________________________
Étape d'indexation manuelle
__________________________________________________________________________________________

In [18]:
with open(file_path, encoding = 'utf-8') as f:
    clusters = pd.read_csv(f, sep=';')
    clusters = clusters[clusters['Visualize (Y/N)'] == 'Y']['Cluster'].tolist()

clusters

['covid-19',
 'influenza',
 'surdoses',
 'cancer',
 'institut national de santé',
 'imagerie',
 'santé mentale et dépendance',
 'centre intégré de santé et de services sociaux',
 'rétention urinaire',
 'gouvernement ministères',
 'imagerie médicale',
 'prévention des infections transmissibles',
 'institut universitaire',
 "virus de l'immunodéficience humaine",
 'consommation de substances psychoactives',
 'réadaptation en déficience physique',
 'échographie cardiaque']

https://inside-machinelearning.com/en/efficient-sentences-embedding-visualization-tsne/

In [19]:
# To speed up the learning process, I suggest to reduce the amount of data. 
to_visualize = df.query('Cluster in @clusters')
to_visualize

,Cluster,Terme,Corpora,Structure syntaxique,Forme lemmatisée,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm,Log Likelihood,TF,DF,TF*IDF,OKapiBM25,TF + DF,tokens,Cluster Name
727,cancer,chum axe de recherche cancer professeur,['chum'],NOM NOM PRP NOM NOM NOM,chum axe de recherche cancer professeur,False,NaN,NaN,NaN,False,531.793003,52,52,0.301786,22.113436,104,"[cancer, chum, professeur, recherche]","Cluster 21, #6 Elements"
797,cancer,diagnostic de cancer,"['chum', 'chuqc', 'iucpq']",NOM PRP NOM,diagnostic de cancer,False,NaN,NaN,NaN,False,545.674438,119,81,0.929090,21.761057,200,"[cancer, diagnostic]","Cluster 21, #6 Elements"
270,cancer,emd serono traitement du cancer,['inesss'],NOM ADJ NOM PRP:det NOM,emd serono traitement du cancer,False,NaN,NaN,NaN,False,820.528144,66,66,0.007977,6.002793,132,"[cancer, serono, traitement]","Cluster 21, #6 Elements"
506,cancer,recherche cancer professeur,['chum'],NOM NOM NOM,recherche cancer professeur,False,NaN,NaN,NaN,False,519.926359,52,52,0.301786,20.333618,104,"[cancer, professeur, recherche]","Cluster 21, #6 Elements"
86,cancer,société canadienne du cancer,"['chuqc', 'ciusss_mcq', 'iucpq', 'laval_sante']",NOM ADJ PRP:det NOM,société canadien du cancer,False,NaN,NaN,NaN,False,1131.563786,206,138,0.644412,12.277896,344,"[canadienne, cancer, société]","Cluster 21, #6 Elements"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,échographie cardiaque,transplantation cardiaque,"['icm', 'iucpq']",NOM ADJ,transplantation cardiaque,True,D016027,Transplantation cardiaque,Heart Transplantation,True,1226.776302,139,54,0.589470,15.005617,193,"[cardiaque, transplantation]","Cluster 7, #12 Elements"
377,échographie cardiaque,échocardiographie de stress,['icm'],NOM PRP NOM,échocardiographie de stress,True,D025401,Échocardiographie de stress,"Echocardiography, Stress",True,149.379796,11,11,0.251353,15.879348,22,"[stress, échocardiographie]","Cluster 7, #12 Elements"
965,échographie cardiaque,échographie cardiaque,"['ciusss_cn', 'ciusss_nordmtl', 'icm', 'iucpq']",NOM ADJ,échographie cardiaque,True,D004452,Échocardiographie,Echocardiography,True,960.497129,237,158,0.720645,10.051189,395,"[cardiaque, échographie]","Cluster 7, #12 Elements"
790,échographie cardiaque,échographie pelvienne échographie rénale échog...,['ciusss_cn'],NOM ADJ NOM ADJ NOM,échographie pelvienne échographie rénal échogr...,False,NaN,NaN,NaN,False,1024.051191,68,68,0.103535,19.911964,136,"[pelvienne, rénale, échographie]","Cluster 7, #12 Elements"


In [20]:
import numpy as np
from sklearn.manifold import TSNE

sentences = to_visualize['Terme'].tolist()
X = list(model.encode(sentences))

X_embedded = TSNE(n_components=2).fit_transform(X)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [21]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=to_visualize.Cluster.values)

In [22]:
df_embeddings = df_embeddings.assign(text=to_visualize.Terme.values)

In [26]:
import plotly.express as px
import plotly

fig = px.scatter(df_embeddings, x='x', y='y', color='label', labels={'color': 'label'}, 
hover_data=['text'], title = 'Quebec Health Care System Terminology Embedding Visualization')
fig.show()

# html file
file_path = '../06-clustering/BERT_communityDetection_visualization_sample.html'
plotly.offline.plot(fig, filename=file_path)


'../06-clustering/BERT_communityDetection_visualization_sample.html'